In [1]:
import os
import json
from os.path import join

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from tqdm.notebook import tqdm
from dn_utils.behavioral_models import load_behavioral_data
from dn_utils.plotting import aligned_imshow_cbar, plot_matrix
from dn_utils.networks import networks_mean, agreemenet_networks
from dn_utils.style import ORANGE, BLUE, POWER_COLORS, lighten_color
from dn_utils.misc import tidy_data
from dn_utils.path import path

from bct.algorithms.clustering import agreement

from statsmodels.stats.multitest import fdrcorrection
from scipy.stats import pearsonr, ttest_rel

%matplotlib inline

/home/kmb/Desktop/Neuroscience/Projects/BONNA_decide_net/code/dn_utils/glm_utils.py:13: FutureWarning: 

 | Starting with Nilearn 0.7.0, all Nistats functionality has been incorporated into Nilearn's stats & reporting modules.
 | Nistats package will no longer be updated or maintained.

  from nistats import design_matrix


In [2]:
path_corrmats = join(path["bsc"], "corrmats")

### Settings

In [3]:
n_nulls = 10_000
gamma_range = np.arange(1.25, 3.5, 0.25)

atlas = "combined_roi"

In [4]:
# Load behavioral data
beh, meta = load_behavioral_data(path["behavioral"], verbose=False)

# Load correlation matrices and metadata
corrmats_aggregated = np.load(
    join(path_corrmats, atlas, "corrmats_aggregated.npy"))
with open(join(path_corrmats, atlas, "corrmats_aggregated.json"), "r") as f:
    corrmats_meta = json.loads(f.read()) 

# Load subject exclusion
df_exclusion = pd.read_csv(
    join(path["nistats"], "exclusion/exclusion.csv"), index_col=0)
ok_index = df_exclusion["ok_all"]

# Load ROI information
df_roi = pd.read_csv(
    join(path_corrmats, atlas, "roi_table_filtered.csv"), index_col=0)
df_roi = df_roi.reset_index()

n_nets = len(df_roi["netName"].unique())

In [5]:
network_names = df_roi["netName"].unique() 
mapping = {net: i for i, net in enumerate(network_names)}

nets = np.array(df_roi["netName"].map(mapping))
nets_unique = mapping.values()
print(nets)

[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  1  1  1  1  1  1  1  1  2  2  2  2  2  2  2  2  2
  2  2  2  2  2  2  2  2  2  2  2  2  2  2  3  3  3  3  3  3  3  3  3  3
  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4
  4  4  4  4  4  4  4  4  4  4  4  5  5  5  5  5  5  5  5  5  5  5  5  5
  5  6  6  6  6  6  6  6  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7
  7  7  7  7  7  8  8  8  8  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9
  9  9  9  9  9  9 10 10 10 10 10 10 10 10 10 11 11 11 11 11 11 11 11 11
 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11
 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 12 12 12
 12 12 12 12 12 12 12 12 12 12 12 13 13 13 13 14 14 14 14 14 14 14 14 14
 14 14 14 14]


### Randomize networks

In [6]:
for gamma in gamma_range:
    print(gamma)

    gamma_str = str(float(gamma)).replace('.', '_')
    path_graph = join(path_corrmats, atlas, "unthr", f"gamma_{gamma_str}")

    # Load graph metrics
    m_aggregated = np.load(join(path_graph, "m_aggregated.npy"))
    q_aggregated = np.load(join(path_graph, "q_aggregated.npy"))
    z_aggregated = np.load(join(path_graph, "z_aggregated.npy"))
    p_aggregated = np.load(join(path_graph, "p_aggregated.npy"))

    n_subjects, n_conditions, n_perr_sign, n_roi = m_aggregated.shape

    d = np.zeros(corrmats_aggregated.shape)
    d_networks = np.zeros(corrmats_aggregated.shape[:3] + (n_nets, n_nets))
    d_networks_null = np.zeros((n_nulls, ) + d_networks.shape)

    network_names = list(df_roi["netName"].unique())

    for sub_idx in tqdm(range(n_subjects)):
        for con_idx in range(n_conditions):
            for perr_sign_idx in range(n_perr_sign):

                m = m_aggregated[sub_idx, con_idx, perr_sign_idx]
                
                # Agreement averaged over LSN pairs
                d_networks[sub_idx, con_idx, perr_sign_idx] = \
                    agreemenet_networks(m, nets, nets_unique)

                for rep in range(n_nulls):
                    np.random.shuffle(m)
                    d_networks_null[rep, sub_idx, con_idx, perr_sign_idx] = \
                        agreemenet_networks(m, nets, nets_unique)
                    
    np.save(join(path_graph, "d_networks_null.npy"), d_networks_null)
    np.save(join(path_graph, "d_networks.npy"), d_networks)
    

1.25


  0%|          | 0/32 [00:00<?, ?it/s]

1.5


  0%|          | 0/32 [00:00<?, ?it/s]

1.75


...

  0%|          | 0/32 [00:00<?, ?it/s]

3.0


  0%|          | 0/32 [00:00<?, ?it/s]

3.25


  0%|          | 0/32 [00:00<?, ?it/s]